# Query Registry
Adversaries may interact with the Windows Registry to gather information about the system, configuration, and installed software.

## Rule Content
```
- title: Query Registry
  id: 970007b7-ce32-49d0-a4a4-fbef016950bd
  status: experimental
  description: Adversaries may interact with the Windows Registry to gather information
    about the system, configuration, and installed software.
  author: Timur Zinniatullin, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1012/T1012.yaml
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image|endswith: \reg.exe
      CommandLine|contains:
      - currentVersion\windows
      - currentVersion\runServicesOnce
      - currentVersion\runServices
      - winlogon\
      - currentVersion\shellServiceObjectDelayLoad
      - currentVersion\runOnce
      - currentVersion\runOnceEx
      - currentVersion\run
      - currentVersion\policies\explorer\run
      - currentcontrolset\services
    condition: selection
  fields:
  - Image
  - CommandLine
  - User
  - LogonGuid
  - Hashes
  - ParentProcessGuid
  - ParentCommandLine
  level: low
  tags:
  - attack.discovery
  - attack.t1012
  - attack.t1007

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\reg.exe AND process_command_line.keyword:(*currentVersion\\windows* OR *currentVersion\\runServicesOnce* OR *currentVersion\\runServices* OR *winlogon\* OR *currentVersion\\shellServiceObjectDelayLoad* OR *currentVersion\\runOnce* OR *currentVersion\\runOnceEx* OR *currentVersion\\run* OR *currentVersion\\policies\\explorer\\run* OR *currentcontrolset\\services*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()